In [3]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import os

# Change the directory to where the datasets are stored
os.chdir('E:\JESIN\DOCUMENTS\scma\A2b')

# Load the datasets
df_ipl = pd.read_csv("IPL_ball_by_ball_updated till 2024.csv", low_memory=False)
salary = pd.read_excel("IPL SALARIES 2024.xlsx")

In [4]:
# Group and aggregate the performance metrics
grouped_data = df_ipl.groupby(['Season', 'Innings No', 'Striker', 'Bowler']).agg({
    'runs_scored': sum, 
    'wicket_confirmation': sum
}).reset_index()

C:\Users\user\AppData\Local\Temp\ipykernel_7884\361222069.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  grouped_data = df_ipl.groupby(['Season', 'Innings No', 'Striker', 'Bowler']).agg({
C:\Users\user\AppData\Local\Temp\ipykernel_7884\361222069.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  grouped_data = df_ipl.groupby(['Season', 'Innings No', 'Striker', 'Bowler']).agg({


In [9]:
grouped_data

,Season,Innings No,Striker,Bowler,runs_scored,wicket_confirmation
0,2007/08,1,A Chopra,DP Vijaykumar,1,0
1,2007/08,1,A Chopra,DW Steyn,1,1
2,2007/08,1,A Chopra,GD McGrath,2,0
3,2007/08,1,A Chopra,PJ Sangwan,6,1
4,2007/08,1,A Chopra,RP Singh,9,0
...,...,...,...,...,...,...
48781,2024,2,YBK Jaiswal,RJW Topley,0,1
48782,2024,2,YBK Jaiswal,SM Curran,6,0
48783,2024,2,YBK Jaiswal,Tilak Varma,5,0
48784,2024,2,YBK Jaiswal,VG Arora,10,1


In [5]:
# Calculate total runs and wickets each year
total_runs_each_year = grouped_data.groupby(['Season', 'Striker'])['runs_scored'].sum().reset_index()
total_wicket_each_year = grouped_data.groupby(['Season', 'Bowler'])['wicket_confirmation'].sum().reset_index()

In [10]:
total_runs_each_year

,Season,Striker,runs_scored
0,2007/08,A Chopra,42
1,2007/08,A Kumble,13
2,2007/08,A Mishra,37
3,2007/08,A Mukund,0
4,2007/08,A Nehra,3
...,...,...,...
2593,2024,Vijaykumar Vyshak,1
2594,2024,WG Jacks,176
2595,2024,WP Saha,135
2596,2024,Washington Sundar,0


In [11]:
total_wicket_each_year

,Season,Bowler,wicket_confirmation
0,2007/08,A Kumble,8
1,2007/08,A Mishra,11
2,2007/08,A Nehra,14
3,2007/08,A Nel,1
4,2007/08,A Symonds,0
...,...,...,...
1924,2024,WG Jacks,2
1925,2024,Washington Sundar,1
1926,2024,YS Chahal,14
1927,2024,Yash Dayal,8


In [6]:
# Function to match names
def match_names(name, names_list):
    match, score = process.extractOne(name, names_list)
    return match if score >= 80 else None  # Use a threshold score of 80


In [7]:
# Matching names for runs
df_salary_runs = salary.copy()
df_runs = total_runs_each_year.copy()
df_salary_runs['Matched_Player'] = df_salary_runs['Player'].apply(lambda x: match_names(x, df_runs['Striker'].tolist()))

# Merge the DataFrames for runs
df_merged_runs = pd.merge(df_salary_runs, df_runs, left_on='Matched_Player', right_on='Striker')

# Subset data for the last three years
df_merged_runs = df_merged_runs.loc[df_merged_runs['Season'].isin(['2021', '2022', '2023'])]

# Perform regression analysis for runs
X = df_merged_runs[['runs_scored']]  # Independent variable(s)
y = df_merged_runs['Rs']  # Dependent variable

# Split the data into training and test sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add a constant to the model (intercept)
X_train_sm = sm.add_constant(X_train)

# Create a statsmodels OLS regression model
model_runs = sm.OLS(y_train, X_train_sm).fit()

# Get the summary of the model
summary_runs = model_runs.summary()
print(summary_runs)

                            OLS Regression Results                            
Dep. Variable:                     Rs   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     15.83
Date:                Sun, 23 Jun 2024   Prob (F-statistic):           0.000100
Time:                        14:40:08   Log-Likelihood:                -1379.8
No. Observations:                 183   AIC:                             2764.
Df Residuals:                     181   BIC:                             2770.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         430.8473     46.111      9.344      

In [8]:
# Matching names for wickets
df_salary_wickets = salary.copy()
df_wickets = total_wicket_each_year.copy()
df_salary_wickets['Matched_Player'] = df_salary_wickets['Player'].apply(lambda x: match_names(x, df_wickets['Bowler'].tolist()))

# Merge the DataFrames for wickets
df_merged_wickets = pd.merge(df_salary_wickets, df_wickets, left_on='Matched_Player', right_on='Bowler')

# Subset data for the last three years
df_merged_wickets = df_merged_wickets.loc[df_merged_wickets['Season'].isin(['2021', '2022', '2023'])]

# Perform regression analysis for wickets
X = df_merged_wickets[['wicket_confirmation']]  # Independent variable(s)
y = df_merged_wickets['Rs']  # Dependent variable

# Split the data into training and test sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add a constant to the model (intercept)
X_train_sm = sm.add_constant(X_train)

# Create a statsmodels OLS regression model
model_wickets = sm.OLS(y_train, X_train_sm).fit()

# Get the summary of the model
summary_wickets = model_wickets.summary()
print(summary_wickets)


                            OLS Regression Results                            
Dep. Variable:                     Rs   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     10.84
Date:                Sun, 23 Jun 2024   Prob (F-statistic):            0.00125
Time:                        14:40:27   Log-Likelihood:                -1094.8
No. Observations:                 145   AIC:                             2194.
Df Residuals:                     143   BIC:                             2200.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 373.6944    

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [12]:
# Perform regression analysis for runs
X = df_merged_runs[['runs_scored']]  # Independent variable(s)
y = df_merged_runs['Rs']  # Dependent variable

# Split the data into training and test sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model_runs = LinearRegression()

# Fit the model on the training data
model_runs.fit(X_train, y_train)

# Make predictions
y_pred = model_runs.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")


R-squared: 0.12271550126860675


In [13]:
# Perform regression analysis for wickets
X = df_merged_wickets[['wicket_confirmation']]  # Independent variable(s)
y = df_merged_wickets['Rs']  # Dependent variable

# Split the data into training and test sets (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model_wickets = LinearRegression()

# Fit the model on the training data
model_wickets.fit(X_train, y_train)

# Make predictions
y_pred = model_wickets.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")


R-squared: 0.014105556424401033
